# Import Packages

In [ ]:
library(opploansanalytics)
load.packages()
library(googlesheets4)

In [ ]:
source('lde-raw-lead-validation.r')

In [ ]:
getAdmethodList() %>% slice(1:20)

# googlesheets4

In [ ]:
sheets_auth()

In [ ]:
getPayloadDiagnostics = function (df, ss.url) {

    effective.sample.size = df %>% nrow()
#         do.call(
#             what = rbind,
#             args = df %>% .$validate.values %>% keep(.p = function (x) {ncol(x) > 1})
#         ) %>% 
#         filter(
#             key == 'email'
#         ) %>% nrow()
    
    valueFormat = function (df, ss.url, effective.sample.size) {
    
        ##  VALUE FORMAT  ##
        do.call(
            what = rbind,
            args = df %>% .$validate.values %>% keep(.p = function (x) {ncol(x) > 1})
        ) %>% 
            mutate(
                value.codes = value.codes %>% map(~ min(.x)) %>% as.character()
            ) %>%
            group_by(
                key,
                value.expected.format = value.codes == 200,
                value.codes
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key,
                value.codes
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Value Types'
            )
        
        ##  Details  ##
        do.call(
            what = rbind,
            args = df %>% .$validate.values %>% keep(.p = function (x) {ncol(x) > 1})
        ) %>% 
            mutate(
                value.codes = value.codes %>% map(~ min(.x)) %>% as.character()
            ) %>%
            group_by(
                key,
                value
            ) %>% 
            filter(
                mean(value.codes != 200) > 0
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key,
                value
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Value Types - Error Details'
            )
        
    }
    valueRange = function (df, ss.url, effective.sample.size) {
    
        ##  VALUE RANGES  ##
        do.call(
            what = rbind,
            args = df %>% .$validate.values %>% keep(.p = function (x) {ncol(x) > 1})
        ) %>% 
            mutate(
                value.in.range = value.in.range %>% as.logical()
            ) %>%
            group_by(
                key,
                value.in.range
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key,
                value.in.range
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Value Ranges'
            )
        
        ##  Details  ##
        do.call(
            what = rbind,
            args = df %>% .$validate.values %>% keep(.p = function (x) {ncol(x) > 1})
        ) %>% 
            mutate(
                value.in.range = value.in.range %>% as.logical()
            ) %>%
            group_by(
                key,
                value
            ) %>% 
            filter(
                mean(value.in.range) <1
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key,
                value
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Value Ranges - Error Details'
            )
    }
    keyMissing = function (df, ss.url, effective.sample.size) {
        ##  MISSING KEYS  ##
        df$keys.missing %>%
            unlist() %>%
            as.data.frame(
                stringsAsFactors = FALSE
            ) %>% 
            select(
                key = "."
            ) %>%
            group_by(
                key
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Keys Missing'
            )
    }
    keyExtra = function (df, ss.url, effective.sample.size) {
        ##  EXTRA KEYS  ##
        df$keys.extra %>%
            unlist() %>%
            as.data.frame(
                stringsAsFactors = FALSE
            ) %>% 
            select(
                key = "."
            ) %>%
            group_by(
                key
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Keys Extra'
            )
    }
    objectMissing = function (df, ss.url, effective.sample.size) {
        ##  MISSING OBJECTS  ##
        do.call(
            what = rbind,
            args = df %>% .$validate.objects %>% keep(.p = function (x) {ncol(x) > 1})
        ) %>% 
            mutate(
                key.codes = key.codes %>% map(~ min(.x)) %>% as.character()
            ) %>%
            group_by(
                key = keys.missing,
                key.missing.allowed = key.codes == 200,
                key.codes
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key,
                key.codes
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Objects Missing'
            )
    }
    objectExtra = function (df, ss.url, effective.sample.size) {
        ##  EXTRA OBJECTS  ##
        df$objects.extra %>%
            unlist() %>%
            as.data.frame(
                stringsAsFactors = FALSE
            ) %>% 
            select(
                key = "."
            ) %>%
            group_by(
                key
            ) %>% 
            summarize(
                n = n(),
                p = n() / effective.sample.size
            ) %>% 
            ungroup() %>% 
            arrange(
                key
            ) %T>%
            sheets_write(
                ss = ss.url,
                sheet = 'Objects Extra'
            )
    }
    
    
    
    valueFormat(df = df, ss.url = ss.url, effective.sample.size = effective.sample.size)
    valueRange(df = df, ss.url = ss.url, effective.sample.size = effective.sample.size)
    keyMissing(df = df, ss.url = ss.url, effective.sample.size = effective.sample.size)
    keyExtra(df = df, ss.url = ss.url, effective.sample.size = effective.sample.size)
    objectMissing(df = df, ss.url = ss.url, effective.sample.size = effective.sample.size)
    objectExtra(df = df, ss.url = ss.url, effective.sample.size = effective.sample.size)
    
    cat('Diagnostics Complete.')
    
}

# Execute

In [ ]:
# lenderedge = 'LenderEdge 4' %>% finalEvaluation(sample.size = 1000)

lenderedge %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=19ExrGhv1xnjFKhHa-G5ndpcOKWB6LVfwijVXqebeLCU'
    )

In [ ]:
# monevo = 'Monevo' %>% finalEvaluation(sample.size = 1000)

monevo %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1od_QpOfyMDwDDoEnVEfO2JptGVKhcj-TUugUJy7z90E'
    )

In [ ]:
# evenfinancial = 'Even Financial 4' %>% finalEvaluation(sample.size = 1000)

evenfinancial %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1DIUP-UDk-ZrbEIda2ZItfFWNojsOnpiI63dPZ-1W1Tc'
    )

In [ ]:
leadgroup = 'LeadGroup' %>% finalEvaluation(sample.size = 1000)

leadgroup %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1_JBkA5bOlObxwNppQMgE9DoHqlZAXfxZWPHh7wuQI1Q'
    )

In [ ]:
quinstreet = 'Quin Street 4' %>% finalEvaluation(sample.size = 1000)

quinstreet %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1kf1N6OhMgmlEX-QTZUssD8Do_LuClc2LDqVBX1q4Ng8'
    )

In [ ]:
# fluent = 'Fluent' %>% finalEvaluation(sample.size = 1000)

# fluent %>%
#     getPayloadDiagnostics(
#         'https://drive.google.com/open?id=1-akBuZHEoLQdEJT_HNv2ECt5Ap4fqmpxfq4rx24j86c'
#     )

In [ ]:
# creditcom = 'Credit.com TWO' %>% finalEvaluation(sample.size = 1000)

# creditcom %>%
#     getPayloadDiagnostics(
#         'https://drive.google.com/open?id=1fMo0ao75nsVmvbQXeBo9d84yn7guxuf5J1JYtmEceoM'
#     )

In [ ]:
leaptheory = 'LeapTheory 4' %>% finalEvaluation(sample.size = 1000)

leaptheory %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1W3zDbcQvawXUn54FQAbnoFZxEyb9hw26yw3PY9hxTHk'
    )

In [ ]:
leadgrouplow = 'Leadgroup Low 4' %>% finalEvaluation(sample.size = 1000)

leadgrouplow %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1RRR9KPOZEEG_3cABuvlEU-Z6fAN7_ah0hv8Zu2NlF28'
    )

In [ ]:
# finder = 'Finder' %>% finalEvaluation(sample.size = 1000)

# finder %>%
#     getPayloadDiagnostics(
#         'https://drive.google.com/open?id=16r-oMy60d9rdpRM6GJJ8AVVXBF41ZE6up5PGtplNfy0'
#     )

In [ ]:
# leadzoom = 'LeadZoom 4' %>% finalEvaluation(sample.size = 1000)

leadzoom %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1xt53FTc-a01sLnFYc_lSVr5h3mUNwoTnUdpW48JGEv0'
    )

In [ ]:
barrelhead = 'Barrelhead 4' %>% finalEvaluation(sample.size = 1000)

barrelhead %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1zDFnyaZOehr7QWRjTPUQlQcKrnMy3CMl8mABVsX9rj4'
    )

In [ ]:
personalloans = 'PersonalLoans.com 4' %>% finalEvaluation(sample.size = 1000)

personalloans %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1IRaYQMSY_5J5jDkj-AiX4AcOmEIhwFRWlJRuY2SD9iQ'
    )

In [ ]:
leadgrouphigh = 'Leadgroup High 4' %>% finalEvaluation(sample.size = 1000)

leadgrouphigh %>%
    getPayloadDiagnostics(
        'https://drive.google.com/open?id=1LbyfovBbITzQDMFbn1iZgxs6iJNzlDpM2AxWkbUKvR4'
    )